# Patient Summary Generator

This notebook generates a comprehensive patient summary from EMBR formated data (optimized FHIR data) using OpenAI's GPT model. Then we display all the EMBR resources used in the summary.

In [6]:
import openai
import json
from dotenv import load_dotenv
import os 
import re
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
with open('..\patients\Clair_Towne.json', 'r') as f:
    patient_data = json.loads(f.read())

In [3]:

# Edit this prompt to fit your needs
prompt = f"""
You are analyzing patient data in EMBR format, an optimized summary of FHIR data. The data includes essential information.

Provide a summary with the following information:
Patient demographics
Conditions
Medications and MedicationRequests
Care plan and goals
Key procedures
Important Encounters and Observations


Only use the data provided, without adding, recommending, or inferring beyond it.
Reference any data with the dates and ID (e.g. EN-129 for Encounter 129).

Patient Data:
{patient_data}
"""

In [4]:
response = openai.chat.completions.create(
    model="gpt-4o-mini", # Not all files can be summarized with gpt-4o, as the tokens are too high
    messages=[
        {"role": "system", "content": "You are a healthcare professional skilled at analyzing and summarizing patient records."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=5000
)

summary = response.choices[0].message.content
display(Markdown(summary))


### Patient Summary

**Patient Demographics:**
- **Name:** Mr. Clair Herschel Towne
- **Gender:** Male
- **Date of Birth:** May 22, 1957
- **Address:** 555 O'Kon Camp Unit 79, Boston, MA, 02467
- **Marital Status:** Married
- **Communication Language:** English (United States)
- **Status:** Living

---

**Conditions:**
1. **Essential hypertension (disorder)** - Confirmed on June 11, 2003 (CN-26)
2. **Osteoarthritis of knee (disorder)** - Confirmed on May 5, 2010 (CN-47)
3. **Ischemic heart disease (disorder)** - Confirmed on August 1, 2012 (CN-59)
4. **Dependent drug abuse (disorder)** - Confirmed on August 28, 2013 (CN-102)
5. **Malignant neoplasm of colon (disorder)** - Confirmed on June 29, 2021 (CN-621)
6. **Chronic pain (finding)** - Confirmed on February 1, 2013 (CN-96)
7. **Gingivitis (disorder)** - Confirmed on August 24, 2016 (CN-226)
8. **Anemia (disorder)** - Confirmed on June 29, 2021 (CN-593)

---

**Medications and Medication Requests:**
- **Active Medications:**
  - Sodium fluoride 0.0272 MG/MG Oral Gel
  - Leucovorin 100 MG Injection
  - 10 ML oxaliplatin 5 MG/ML Injection
- **Medication Requests:**
  - Naproxen sodium 220 MG Oral Tablet (MR-49)
  - Hydrochlorothiazide 25 MG Oral Tablet (MR-243, stopped)
  - Various antihypertensive and antiplatelet medications.

---

**Care Plan and Goals:**
1. **Lifestyle education regarding hypertension** - Active since June 11, 2003 (CP-28)
2. **Musculoskeletal care (regime/therapy)** - Active since May 5, 2010 (CP-53)
3. **Cancer care plan** - Completed on August 15, 2023 (CP-623)
4. **Drug addiction counseling and therapy** - Active since September 25, 2017 (CP-315)

---

**Key Procedures:**
- **Colonoscopy** - Completed on June 23, 2021 (PR-594)
- **Partial resection of colon** - Completed on July 9, 2021 (PR-658)
- **Combined chemotherapy and radiation therapy** - Multiple instances including August 7, 2021 (PR-695) and November 7, 2021 (PR-872)

---

**Important Encounters and Observations:**
1. **Encounter for drug overdose** - Emergency room admission on August 4, 2023 (EN-1108)
2. **General examinations** - Multiple encounters including October 4, 2023 (EN-1125)
3. **Pain severity assessments** - Documented pain levels ranging from 0 to 8 on a 10-point scale during various encounters.

**Key Observations:**
- **Body Height:** 183.1 cm
- **Body Weight:** 101.1 kg
- **Blood Pressure:** Fluctuates between systolic readings of 115 mmHg and 134 mmHg.
- **Cholesterol Levels:** Varied but highest reported total cholesterol of 216.11 mg/dL (2024).
- **GAD-7 Scores:** Indicating anxiety levels, including a score of 11 on September 22, 2021 (OB-236).

---

This summary captures essential aspects of Mr. Clair Herschel Towne's health status, including demographics, medical conditions, medication management, care plans, key procedures, and significant encounters and observations over time.

***
***
# Resource Referenced in Summary 

In the previous section, we generated a comprehensive patient summary by processing EMBR file and extracting relevant medical details. Now, let's examine the resources used to generate the summary.

# Testability and Explainability
Having references enhances the testability of your Generative AI results, allowing you to verify against the ground truth.

In [12]:
matches = re.findall(r'[A-Z]{2}-\d+', summary)
for type in patient_data:
    for resource in patient_data[type]:
        if resource['id'] in matches:
            display(Markdown(f"### {type} - {resource.get('id')}"))
            details = []
            for key in resource:
                if key != 'id':
                    if isinstance(resource[key], list):
                        details.append(f"- **{key}**:")
                        for item in resource[key]:
                            details.append(f"  - {item}")
                    else:
                        details.append(f"- **{key}**: {resource[key]}")
            display(Markdown('\n'.join(details)))
            display(Markdown("---"))

### Encounter - EN-1108

- **date**: 2023-08-04T09:21:45
- **ref**:
- **code**:
  - Emergency room admission
  - Drug overdose
  - emergency
- **stat**: finished
- **provider**: performer Dr. Wilson Turner
- **facility**: KATZMAN FAMILY CENTER FOR LIVING
- **endDate**: 2023-08-04T12:21:45

---

### Encounter - EN-1125

- **date**: 2023-10-04T10:39:02
- **ref**:
- **code**:
  - General examination of patient
  - ambulatory
- **stat**: finished
- **provider**: performer Dr. Maricruz Considine
- **facility**: NORTH END COMMUNITY HEALTH COMMITTEE INC
- **endDate**: 2023-10-04T11:10:17

...

- **date**: 2021-11-07T16:01:57
- **ref**:
  - EN-843
  - CN-621
- **code**:
  - Combined chemotherapy and radiation therapy
- **stat**: completed
- **endDate**: 2021-11-07T16:16:57

---

### Observation - OB-236

- **date**: 2016-08-24T11:46:19
- **ref**:
  - EN-224
- **code**:
  - Generalized anxiety disorder 7 item (GAD-7) total score [Reported.PHQ] is 4 {score}
- **stat**: final
- **category**: Survey

---